# Cuaderno 01: Generación de Datasets y Embeddings

### Objetivo 🎯
Este cuaderno es el punto de partida fundamental de nuestro proyecto. Su objetivo es construir los **datasets paralelos** (auditivo y visual) y las **representaciones numéricas (embeddings)** que servirán de base para el entrenamiento de los modelos.

### Flujo de Trabajo  Workflow ⚙️

1.  **Configuración e Instalación**: Definiremos los parámetros globales, las listas de unidades lingüísticas (fonemas y palabras) y las rutas de salida. También instalaremos las librerías necesarias.
2.  **Generación de Audio para Fonemas**: Crearemos archivos `.wav` para cada fonema/grafema de nuestras listas utilizando la librería `gTTS`.
3.  **Generación de Audio para Palabras**: Cargaremos listas de palabras desde archivos de diccionario y generaremos sus correspondientes archivos `.wav`.
4.  **Generación del Dataset Visual (EMNIST)**: Descargaremos y extraeremos imágenes de letras (`.png`) del dataset EMNIST que correspondan a nuestros grafemas.
5.  **Extracción de Embeddings Auditivos (Wav2Vec2)**: Procesaremos **todos** los archivos `.wav` generados (tanto fonemas como palabras) con el modelo `Wav2Vec2` para convertirlos en secuencias de vectores (embeddings) y guardarlos como archivos `.npy`.
6.  **Resumen**: Al final de cada paso, se mostrará una tabla de resumen con el estado de la generación de archivos.

## Paso 1: Configuración e Instalación

Primero, instalamos y luego importamos todas las librerías que usaremos en el cuaderno. Separamos la configuración de los parámetros y rutas para mantener el código limpio y modular.

In [25]:
%pip install gTTS pydub torch torchvision matplotlib transformers librosa protobuf pandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [26]:
# Librerías estándar de Python
import os
from pathlib import Path

# Librerías para generación de audio
from gtts import gTTS
from pydub import AudioSegment
import librosa

# Librerías para manejo de imágenes y datasets
import torch
import torchvision
from torchvision.datasets import EMNIST
from PIL import Image

# Librerías para modelos de audio y arrays numéricos
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
import numpy as np

# Librería para manejo y visualización de datos tabulares
import pandas as pd

## Paso 1.2: Configuración, Rutas y Parámetros

Con las dependencias instaladas, el siguiente paso es configurar nuestro entorno de trabajo. Esta celda se encarga de:

* **Importar las librerías** necesarias para el resto del cuaderno.
* **Definir las rutas de archivos**: Se establece una estructura de carpetas organizada para los datos brutos (`01_raw`) y los datos procesados (`02_processed`), separando los datasets auditivos y visuales.
* **Establecer las listas de fonemas/grafemas**: Se definen las unidades fonéticas y gráficas para español e inglés que se generarán en las Partes A y B.
* **Fijar parámetros de procesamiento**: Como el número máximo de imágenes a extraer por cada grafema (`MAX_IMAGES_PER_LETTER`).

In [27]:
# --- Parámetros Globales del Experimento ---
MAX_IMAGES_PER_LETTER = 50
LANGUAGES = ['es', 'en']
WAV2VEC2_MODEL_NAME = "facebook/wav2vec2-large-xlsr-53"
TARGET_SAMPLE_RATE = 16000

# --- Rutas del Proyecto ---
project_root = Path.cwd().parent

# Directorios de datos crudos (entrada)
raw_emnist_dir = project_root / "data/01_raw/emnist"
dictionaries_dir = project_root / "data/01_raw/dictionaries" # Ruta para los diccionarios

# Directorios de datos procesados (salida)
output_audio_dir = project_root / "data/02_processed/phoneme_audio"
output_word_audio_dir = project_root / "data/02_processed/word_audio"
output_visual_dir = project_root / "data/02_processed/grapheme_images"
output_embedding_dir = project_root / "data/02_processed/wav2vec2_embeddings"
output_word_embedding_dir = project_root / "data/02_processed/word_embeddings"

# Crear todos los directorios de salida
for p in [output_audio_dir, output_word_audio_dir, output_visual_dir, output_embedding_dir, output_word_embedding_dir, raw_emnist_dir, dictionaries_dir]:
    p.mkdir(parents=True, exist_ok=True)
    
# --- Listas de Fonemas/Grafemas ---
PHONEMES_GRAPHEMES = {
    'es': ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z'],
    'en': ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
}

# --- Carga de Listas de Palabras desde Diccionarios ---
WORDS = {}
for lang in LANGUAGES:
    dict_path = dictionaries_dir / f"{lang}_words.txt"
    try:
        with open(dict_path, 'r', encoding='utf-8') as f:
            WORDS[lang] = [line.strip() for line in f if line.strip()]
        print(f"Diccionario para '{lang.upper()}' cargado con {len(WORDS[lang])} palabras.")
    except FileNotFoundError:
        print(f"ADVERTENCIA: No se encontró el archivo de diccionario en '{dict_path}'. La generación de audio para palabras de '{lang.upper()}' será omitida.")
        WORDS[lang] = []

Diccionario para 'ES' cargado con 100 palabras.
Diccionario para 'EN' cargado con 100 palabras.


## Paso 2: Generación de Audio para Fonemas (TTS)

Primero, definimos una función de ayuda que mostrará los resúmenes en formato de tabla. Luego, generamos los archivos de audio para las unidades fonéticas/grafémicas básicas.

In [28]:
def display_summary(summary_log, task_title=""):
    """Convierte un log de generación en un DataFrame y muestra un resumen."""
    print(f"\n--- {task_title} ---")
    if not summary_log:
        print("No se procesó ningún archivo.")
        return
        
    df = pd.DataFrame(summary_log)
    
    print("Resumen de la operación:")
    print(df['estado'].value_counts().to_string())
    print("\nDetalle completo:")
    display(df)

In [29]:
def generate_audio_files(item_list, lang_code, output_path):
    """Sintetiza archivos WAV y devuelve un log de la operación."""
    output_path.mkdir(parents=True, exist_ok=True)
    summary_log = []
    
    for item in item_list:
        wav_filepath = output_path / f"{item}.wav"
        if wav_filepath.exists():
            summary_log.append({'ítem': item, 'ruta': wav_filepath, 'estado': 'Omitido'})
            continue
        
        try:
            tts = gTTS(text=item, lang=lang_code, slow=True)
            mp3_temp_path = output_path / "_temp.mp3"
            tts.save(mp3_temp_path)
            
            audio = AudioSegment.from_mp3(mp3_temp_path)
            audio = audio.set_channels(1).set_frame_rate(16000)
            audio.export(wav_filepath, format="wav")
            
            os.remove(mp3_temp_path)
            summary_log.append({'ítem': item, 'ruta': wav_filepath, 'estado': 'Creado'})
        except Exception as e:
            summary_log.append({'ítem': item, 'ruta': wav_filepath, 'estado': f'Error: {e}'})
            if 'mp3_temp_path' in locals() and mp3_temp_path.exists():
                os.remove(mp3_temp_path)
    return summary_log

In [30]:
# Ejecutar la generación y mostrar resumen para cada idioma
for lang in LANGUAGES:
    log = generate_audio_files(
        item_list=PHONEMES_GRAPHEMES[lang],
        lang_code=lang,
        output_path=output_audio_dir / lang
    )
    display_summary(log, f"Resumen de Audio para Fonemas ({lang.upper()})")


--- Resumen de Audio para Fonemas (ES) ---
Resumen de la operación:
estado
Creado    25

Detalle completo:


,ítem,ruta,estado
0,a,/home/daniel/Proyectos/phonological-awareness/...,Creado
1,b,/home/daniel/Proyectos/phonological-awareness/...,Creado
2,c,/home/daniel/Proyectos/phonological-awareness/...,Creado
3,d,/home/daniel/Proyectos/phonological-awareness/...,Creado
4,e,/home/daniel/Proyectos/phonological-awareness/...,Creado
5,f,/home/daniel/Proyectos/phonological-awareness/...,Creado
6,g,/home/daniel/Proyectos/phonological-awareness/...,Creado
7,h,/home/daniel/Proyectos/phonological-awareness/...,Creado
8,i,/home/daniel/Proyectos/phonological-awareness/...,Creado
9,j,/home/daniel/Proyectos/phonological-awareness/...,Creado



--- Resumen de Audio para Fonemas (EN) ---
Resumen de la operación:
estado
Creado    25

Detalle completo:


,ítem,ruta,estado
0,a,/home/daniel/Proyectos/phonological-awareness/...,Creado
1,b,/home/daniel/Proyectos/phonological-awareness/...,Creado
2,c,/home/daniel/Proyectos/phonological-awareness/...,Creado
3,d,/home/daniel/Proyectos/phonological-awareness/...,Creado
4,e,/home/daniel/Proyectos/phonological-awareness/...,Creado
5,f,/home/daniel/Proyectos/phonological-awareness/...,Creado
6,g,/home/daniel/Proyectos/phonological-awareness/...,Creado
7,h,/home/daniel/Proyectos/phonological-awareness/...,Creado
8,i,/home/daniel/Proyectos/phonological-awareness/...,Creado
9,j,/home/daniel/Proyectos/phonological-awareness/...,Creado


## Paso 3: Generación de Audio para Palabras (TTS)

Ahora, aplicamos la misma función de generación de audio a las listas de palabras que cargamos desde los archivos de diccionario.

In [31]:
# Ejecutar la generación para las palabras de cada idioma
for lang in LANGUAGES:
    if WORDS[lang]: # Solo ejecutar si la lista de palabras no está vacía
        log = generate_audio_files(
            item_list=WORDS[lang],
            lang_code=lang,
            output_path=output_word_audio_dir / lang
        )
        display_summary(log, f"Resumen de Audio para Palabras ({lang.upper()})")


--- Resumen de Audio para Palabras (ES) ---
Resumen de la operación:
estado
Creado    100

Detalle completo:


,ítem,ruta,estado
0,casa,/home/daniel/Proyectos/phonological-awareness/...,Creado
1,perro,/home/daniel/Proyectos/phonological-awareness/...,Creado
2,gato,/home/daniel/Proyectos/phonological-awareness/...,Creado
3,sol,/home/daniel/Proyectos/phonological-awareness/...,Creado
4,luna,/home/daniel/Proyectos/phonological-awareness/...,Creado
...,...,...,...
95,querer,/home/daniel/Proyectos/phonological-awareness/...,Creado
96,poder,/home/daniel/Proyectos/phonological-awareness/...,Creado
97,hacer,/home/daniel/Proyectos/phonological-awareness/...,Creado
98,ir,/home/daniel/Proyectos/phonological-awareness/...,Creado



--- Resumen de Audio para Palabras (EN) ---
Resumen de la operación:
estado
Creado    100

Detalle completo:


,ítem,ruta,estado
0,house,/home/daniel/Proyectos/phonological-awareness/...,Creado
1,dog,/home/daniel/Proyectos/phonological-awareness/...,Creado
2,cat,/home/daniel/Proyectos/phonological-awareness/...,Creado
3,sun,/home/daniel/Proyectos/phonological-awareness/...,Creado
4,moon,/home/daniel/Proyectos/phonological-awareness/...,Creado
...,...,...,...
95,want,/home/daniel/Proyectos/phonological-awareness/...,Creado
96,can,/home/daniel/Proyectos/phonological-awareness/...,Creado
97,do,/home/daniel/Proyectos/phonological-awareness/...,Creado
98,go,/home/daniel/Proyectos/phonological-awareness/...,Creado


## Paso 4: Generación del Dataset Visual (EMNIST)

Para la contraparte visual, usamos el dataset EMNIST para obtener imágenes de letras individuales. Esto nos servirá para entrenar la vía visual en el Cuaderno 03.

In [32]:
def generate_visual_dataset(phoneme_list, emnist_dataset, output_path, max_images):
    """Filtra EMNIST, guarda imágenes y devuelve un log de la operación."""
    class_to_char = {i: chr(ord('a') + i) for i in range(26)}
    char_to_class = {c: i for i, c in class_to_char.items()}
    summary_log = []

    single_char_graphemes = sorted([p for p in phoneme_list if len(p) == 1 and p.lower() in char_to_class])

    images_by_label = {i: [] for i in range(len(emnist_dataset.classes))}
    for image, label in emnist_dataset:
        images_by_label[label - 1].append(image)

    for grapheme in single_char_graphemes:
        class_idx = char_to_class.get(grapheme.lower())
        if class_idx is not None:
            grapheme_dir = output_path / grapheme
            grapheme_dir.mkdir(parents=True, exist_ok=True)
            
            num_existing = len(list(grapheme_dir.glob("*.png")))
            if num_existing >= max_images:
                summary_log.append({'grafema': grapheme, 'archivos_generados': 0, 'estado': 'Límite alcanzado'})
                continue

            images_to_save = images_by_label[class_idx][:max_images - num_existing]
            
            for j, img_obj in enumerate(images_to_save):
                img = img_obj.transpose(Image.Transpose.ROTATE_270).transpose(Image.Transpose.FLIP_LEFT_RIGHT)
                img.save(grapheme_dir / f"{grapheme}_{num_existing + j}.png")
            
            summary_log.append({'grafema': grapheme, 'archivos_generados': len(images_to_save), 'estado': 'Creado'})
        else:
            summary_log.append({'grafema': grapheme, 'archivos_generados': 0, 'estado': 'No encontrado en EMNIST'})
    return summary_log

In [33]:
# Descargar EMNIST una sola vez
print("\n--- Descargando y preparando el dataset EMNIST ---")
emnist_dataset = EMNIST(root=raw_emnist_dir, split='letters', download=True)

# Ejecutar la generación para cada idioma
for lang in LANGUAGES:
    log = generate_visual_dataset(
        phoneme_list=PHONEMES_GRAPHEMES[lang],
        emnist_dataset=emnist_dataset,
        output_path=output_visual_dir / lang,
        max_images=MAX_IMAGES_PER_LETTER
    )
    display_summary(log, f"Resumen de Imágenes de Grafemas ({lang.upper()})")


--- Descargando y preparando el dataset EMNIST ---

--- Resumen de Imágenes de Grafemas (ES) ---
Resumen de la operación:
estado
Creado    25

Detalle completo:


,grafema,archivos_generados,estado
0,a,50,Creado
1,b,50,Creado
2,c,50,Creado
3,d,50,Creado
4,e,50,Creado
5,f,50,Creado
6,g,50,Creado
7,h,50,Creado
8,i,50,Creado
9,j,50,Creado



--- Resumen de Imágenes de Grafemas (EN) ---
Resumen de la operación:
estado
Creado    25

Detalle completo:


,grafema,archivos_generados,estado
0,a,50,Creado
1,b,50,Creado
2,c,50,Creado
3,d,50,Creado
4,e,50,Creado
5,f,50,Creado
6,g,50,Creado
7,h,50,Creado
8,i,50,Creado
9,j,50,Creado


## Paso 5: Extracción de Embeddings Auditivos (Wav2Vec2)

Ahora que tenemos todos los archivos de audio (fonemas y palabras), los convertimos en **embeddings** usando `Wav2Vec2`. Este paso es crucial, ya que transforma el sonido en una representación vectorial que los modelos de deep learning pueden procesar.

In [34]:
# --- Carga del Modelo y Extractor de Características ---
print("Cargando el modelo Wav2Vec2... (esto puede tardar la primera vez)")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_MODEL_NAME)
model_wav2vec = Wav2Vec2Model.from_pretrained(WAV2VEC2_MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_wav2vec.to(device)
print(f"Modelo cargado y movido al dispositivo: {device}")

Cargando el modelo Wav2Vec2... (esto puede tardar la primera vez)
Modelo cargado y movido al dispositivo: cuda


In [35]:
def extract_embeddings(audio_dir, embedding_dir):
    """Procesa una carpeta de archivos .wav, guarda sus embeddings y devuelve un log."""
    embedding_dir.mkdir(parents=True, exist_ok=True)
    audio_files = sorted(list(audio_dir.glob("*.wav")))
    summary_log = []
    
    for path in audio_files:
        label = path.stem
        embedding_path = embedding_dir / f"{label}.npy"
        if embedding_path.exists():
            summary_log.append({'ítem': label, 'ruta': embedding_path, 'estado': 'Omitido'})
            continue
        try:
            waveform, _ = librosa.load(path, sr=TARGET_SAMPLE_RATE)
            inputs = feature_extractor(waveform, sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt", padding=True).input_values.to(device)
            
            with torch.no_grad():
                outputs = model_wav2vec(inputs)
            
            sequence_embedding = outputs.last_hidden_state.squeeze(0).cpu().numpy()
            np.save(embedding_path, sequence_embedding)
            summary_log.append({'ítem': label, 'ruta': embedding_path, 'estado': 'Creado'})
        except Exception as e:
            summary_log.append({'ítem': label, 'ruta': embedding_path, 'estado': f'Error: {e}'})
    return summary_log

In [36]:
# --- Ejecutar Extracción para Todos los Datasets Auditivos ---
# Fonemas
for lang in LANGUAGES:
    log = extract_embeddings(
        audio_dir=output_audio_dir / lang,
        embedding_dir=output_embedding_dir / lang
    )
    display_summary(log, f"Resumen de Embeddings para Fonemas ({lang.upper()})")

# Palabras
for lang in LANGUAGES:
    if WORDS[lang]:
        log = extract_embeddings(
            audio_dir=output_word_audio_dir / lang,
            embedding_dir=output_word_embedding_dir / lang
        )
        display_summary(log, f"Resumen de Embeddings para Palabras ({lang.upper()})")


--- Resumen de Embeddings para Fonemas (ES) ---
Resumen de la operación:
estado
Creado    25

Detalle completo:


,ítem,ruta,estado
0,a,/home/daniel/Proyectos/phonological-awareness/...,Creado
1,b,/home/daniel/Proyectos/phonological-awareness/...,Creado
2,c,/home/daniel/Proyectos/phonological-awareness/...,Creado
3,d,/home/daniel/Proyectos/phonological-awareness/...,Creado
4,e,/home/daniel/Proyectos/phonological-awareness/...,Creado
5,f,/home/daniel/Proyectos/phonological-awareness/...,Creado
6,g,/home/daniel/Proyectos/phonological-awareness/...,Creado
7,h,/home/daniel/Proyectos/phonological-awareness/...,Creado
8,i,/home/daniel/Proyectos/phonological-awareness/...,Creado
9,j,/home/daniel/Proyectos/phonological-awareness/...,Creado



--- Resumen de Embeddings para Fonemas (EN) ---
Resumen de la operación:
estado
Creado    25

Detalle completo:


,ítem,ruta,estado
0,a,/home/daniel/Proyectos/phonological-awareness/...,Creado
1,b,/home/daniel/Proyectos/phonological-awareness/...,Creado
2,c,/home/daniel/Proyectos/phonological-awareness/...,Creado
3,d,/home/daniel/Proyectos/phonological-awareness/...,Creado
4,e,/home/daniel/Proyectos/phonological-awareness/...,Creado
5,f,/home/daniel/Proyectos/phonological-awareness/...,Creado
6,g,/home/daniel/Proyectos/phonological-awareness/...,Creado
7,h,/home/daniel/Proyectos/phonological-awareness/...,Creado
8,i,/home/daniel/Proyectos/phonological-awareness/...,Creado
9,j,/home/daniel/Proyectos/phonological-awareness/...,Creado



--- Resumen de Embeddings para Palabras (ES) ---
Resumen de la operación:
estado
Creado    100

Detalle completo:


,ítem,ruta,estado
0,adiós,/home/daniel/Proyectos/phonological-awareness/...,Creado
1,agua,/home/daniel/Proyectos/phonological-awareness/...,Creado
2,ahora,/home/daniel/Proyectos/phonological-awareness/...,Creado
3,allí,/home/daniel/Proyectos/phonological-awareness/...,Creado
4,alto,/home/daniel/Proyectos/phonological-awareness/...,Creado
...,...,...,...
95,ver,/home/daniel/Proyectos/phonological-awareness/...,Creado
96,verde,/home/daniel/Proyectos/phonological-awareness/...,Creado
97,vida,/home/daniel/Proyectos/phonological-awareness/...,Creado
98,viejo,/home/daniel/Proyectos/phonological-awareness/...,Creado



--- Resumen de Embeddings para Palabras (EN) ---
Resumen de la operación:
estado
Creado    100

Detalle completo:


,ítem,ruta,estado
0,after,/home/daniel/Proyectos/phonological-awareness/...,Creado
1,all,/home/daniel/Proyectos/phonological-awareness/...,Creado
2,always,/home/daniel/Proyectos/phonological-awareness/...,Creado
3,bad,/home/daniel/Proyectos/phonological-awareness/...,Creado
4,before,/home/daniel/Proyectos/phonological-awareness/...,Creado
...,...,...,...
95,woman,/home/daniel/Proyectos/phonological-awareness/...,Creado
96,work,/home/daniel/Proyectos/phonological-awareness/...,Creado
97,world,/home/daniel/Proyectos/phonological-awareness/...,Creado
98,yellow,/home/daniel/Proyectos/phonological-awareness/...,Creado


## Paso 6: Conclusión y Próximos Pasos

Hemos generado con éxito todos los datos de entrada necesarios para nuestros modelos, y hemos verificado la creación de cada conjunto de archivos mediante tablas de resumen.

**Activos Generados:**
-   ✅ **Archivos de Audio**: `.wav` para fonemas y palabras.
-   ✅ **Embeddings Auditivos**: Archivos `.npy` para fonemas y palabras.
-   ✅ **Imágenes de Grafemas**: Carpetas de imágenes `.png` para las letras.

Con estos datos listos y validados, podemos proceder a los cuadernos de entrenamiento.